In [3]:
import numpy as np
import pandas as pd

This kernel does not predict anything. Instead, it tries to address a problem that many newcomers to this competition may have: **The training data is *huge*, simply reading the full csv at once may be too much**. If you want to play around with the data on a local machine, you need something better. Fortunately, the data can be encoded much more efficiently:
* time_to_failure is *for all practical purposes* (see explanation below) linear and does not need to be saved. It can be generated by numpy.linspace as needed
* acoustic_data can be saved in a compressed npz file

The resulting file will be ~400MB in total, can be easily saved in the cloud, contains the full acoustic_data and can be loaded in ~ 10 sec on most machines.

**Run this notebook as a Kaggle kernel and download the output files *train_acoustic_data.npz* and *train_info.csv* .**

EDIT: 2019-04-17: added the same procedure for the test data as well.

In [ ]:
%time df_train = pd.read_csv('../input/train.csv', dtype = {'acoustic_data': np.int16, 'time_to_failure': np.float32} ) # float32 is enough :)

Yes, that took forever.

There are 17 "earthquake periods" (the last one ends long before its quake). Let's find the beginning of each period:

In [ ]:
ttf = df_train['time_to_failure'].values
index_start = np.nonzero(np.diff(ttf) > 0)[0] + 1
index_start = np.insert(index_start, 0, 0) # insert 1st period start manually
print(index_start)

How many data points are in each period?

In [ ]:
chunk_length = np.diff(np.append(index_start, df_train.shape[0]))
print(chunk_length)

What are the time_to_failure values of the 1st and last data point in each period?

In [ ]:
t_start = ttf[index_start]
t_end = ttf[index_start + chunk_length - 1]
print('t_start =', t_start)
print('t_end   =', t_end)

Assuming that the acoustic_data is contiguous (that's probably not strictly true, see the discussions, but it doesn't really matter anyway), what is the length of the time steps between data points in each period? The values should be approximately equal, and they are:

In [ ]:
dt_step = (t_start - t_end) / chunk_length
dt_step

Now it gets interesting. If we approximate the time_to_failure column for each period with a numpy.linspace from t_start to t_end, what is the maximal difference (i.e. error) between this approximation and the original data? Result: 1.15 ms. **This is 3 orders of magnitude below the prediction accuracy that we can reasonably hope to achieve (O(1 sec)). We don't need the original time_to_failure column anymore.**

In [ ]:
linsp_diff_max = []
for i in range(len(index_start)):
    ttf_orig  = ttf[index_start[i] : index_start[i] + chunk_length[i]]
    ttf_linsp = np.linspace(t_start[i], t_end[i], chunk_length[i])
    linsp_diff_max.append(np.abs(ttf_orig - ttf_linsp).max())
linsp_diff_max = np.array(linsp_diff_max)
print(linsp_diff_max)
print('Max error =', linsp_diff_max.max())

Let's collect all of this into a dataframe which will be saved (we will need it again to generate time_to_failure!)

In [ ]:
df_train_info = pd.DataFrame({
    'index_start':index_start,
    'chunk_length':chunk_length,
    't_start':t_start,
    't_end':t_end,
    'dt_step':dt_step,
    'linsp_diff_max':linsp_diff_max
})
df_train_info.to_csv('train_info.csv', index=False)
df_train_info

Let's save the acoustic data in compressed form. This is only done once. Yep, we're below 400MB:

In [ ]:
%time np.savez_compressed('train_acoustic_data.npz', acoustic_data=df_train['acoustic_data'].values)
%ls -lh

Now we're done. From now on, you only need *train_acoustic_data.npz* , *train_info.csv* and a function to extract a quake period and generate the time_to_failure data:

In [ ]:
%%time
df_train_info = pd.read_csv('train_info.csv')
ac_data = np.load('train_acoustic_data.npz')['acoustic_data']

def get_quake_period(i):
    index_start, chunk_length = df_train_info['index_start'][i], df_train_info['chunk_length'][i]
    t_start, t_end = df_train_info['t_start'][i], df_train_info['t_end'][i]
    ac_data_period = ac_data[ index_start : index_start + chunk_length ]
    ttf_data_period = np.linspace(t_start, t_end, chunk_length, dtype=np.float32)
    return ac_data_period, ttf_data_period

Let's extract period #3:

In [ ]:
get_quake_period(3)

That's all. Happy Predicting!

Well, almost: let's bring the huge amount of litte test files into a manageable format as well:

In [2]:
df_ssub = pd.read_csv('../input/sample_submission.csv')
df_ssub.tail()

,seg_id,time_to_failure
2619,seg_ff4236,0
2620,seg_ff7478,0
2621,seg_ff79d9,0
2622,seg_ffbd6a,0
2623,seg_ffe7cc,0


Pack the 2624 test segments into a 2624 x 150000 numpy array and save it:

In [15]:
ac_data = []

for fname in df_ssub['seg_id'].values:
    ac_data.append(pd.read_csv('../input/test/' + fname + '.csv').acoustic_data.values.astype(np.int16))
    
ac_data = np.array(ac_data)

np.savez_compressed('test_acoustic_data.npz', acoustic_data=ac_data)

print('test data shape :', ac_data.shape)